#Homework with Yelp reviews data

##Introduction
- This assignment uses a small subset of the data from Kaggle's Yelp Business Rating Prediction competition.
**At the heart of any recommender system is an algorithm to predict ratings.  Contestants are asked to predict the users’ future ratings of businesses. Participants will create a model to predict the rating a user would assign to a business. Models will be graded on accuracy using the root mean squared error metric. See the Evaluation page for more details**

**Description of the data**:
1. *yelp.csv* contains the dataset. It is stored in the course repository (in the data directory), so there is no need to download anything from the Kaggle website.

2. *Each observation (row)* in this dataset is a review of a particular business by a particular user.

3.  The *stars column* is the number of stars (1 through 5) assigned by the reviewer to the business. (Higher stars is better.) In other words, it is the rating of the business by the person who wrote the review.

4. The *text column* is the text of the review.

5. The *cool column* is the number of "cool" votes this review received from other Yelp users. All reviews start with 0 "cool" votes, and there is no limit to how many "cool" votes a review can receive. In other words, it is a rating of the review itself, not a rating of the business.

6. The *useful and funny* columns are similar to the cool column.

**Goal**: Predict the star rating of a review using only the review text. (We will not be using the cool, funny, or useful columns.)

**Tip**: After each task, I recommend that you check the shape and the contents of your objects, to confirm that they match your expectations.

In [141]:
# Verify in the correct directory
import os
print os.getcwd()

/Users/caboe/Dropbox/DataSchool/Untitled Folder


In [142]:
# Import the data from the github repo
# This requires importing pandas
import pandas as pd
URL = 'https://raw.githubusercontent.com/dataschool/MLtext2/master/data/yelp.csv?token=AEv2W1BCiQQ1BDnLNLKueYQk6kBwkfb4ks5XFCmAwA%3D%3D'
posts = pd.read_csv(URL)

In [143]:
# These URLs worked in a pinch when I couldn't access the private repos
# posts = pd.read_csv('https://github.com/dataschool/MLtext2/blob/master/data/yelp.csv?raw=true')
#url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/sms.tsv'
#sms = pd.read_table(url)

In [144]:
# Examine the DF
posts.head(5)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [145]:
# Examine the last few rows in the df
posts.tail(5)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
9995,VY_tvNUCCXGXQeSvJl757Q,2012-07-28,Ubyfp2RSDYW0g7Mbr8N3iA,3,First visit...Had lunch here today - used my G...,review,_eqQoPtQ3e3UxLE4faT6ow,1,2,0
9996,EKzMHI1tip8rC1-ZAy64yg,2012-01-18,2XyIOQKbVFb6uXQdJ0RzlQ,4,Should be called house of deliciousness!\n\nI ...,review,ROru4uk5SaYc3rg8IU7SQw,0,0,0
9997,53YGfwmbW73JhFiemNeyzQ,2010-11-16,jyznYkIbpqVmlsZxSDSypA,4,I recently visited Olive and Ivy for business ...,review,gGbN1aKQHMgfQZkqlsuwzg,0,0,0
9998,9SKdOoDHcFoxK5ZtsgHJoA,2012-12-02,5UKq9WQE1qQbJ0DJbc-B6Q,2,My nephew just moved to Scottsdale recently so...,review,0lyVoNazXa20WzUyZPLaQQ,0,0,0
9999,pF7uRzygyZsltbmVpjIyvw,2010-10-16,vWSmOhg2ID1MNZHaWapGbA,5,4-5 locations.. all 4.5 star average.. I think...,review,KSBFytcdjPKZgXKQnYQdkA,0,0,0


In [146]:
# Check the value counts on the classes we will be predicting with our model
posts.stars.value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [147]:
type(posts.stars.value_counts())

pandas.core.series.Series

In [148]:
posts.stars.value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [149]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
business_id    10000 non-null object
date           10000 non-null object
review_id      10000 non-null object
stars          10000 non-null int64
text           10000 non-null object
type           10000 non-null object
user_id        10000 non-null object
cool           10000 non-null int64
useful         10000 non-null int64
funny          10000 non-null int64
dtypes: int64(4), object(6)
memory usage: 781.3+ KB


###Task 2: Create a new DataFrame that only contains the 5-star and 1-star reviews.

In [150]:
# Create a dataframe with only 1 and 5 star reviews.
fives_ones = posts[(posts['stars'] > 4) |(posts['stars'] < 2)]

In [151]:
# check the shape of the new daatframe
fives_ones.shape

(4086, 10)

In [152]:
# look at head
fives_ones.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
6,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,7,7,4


In [153]:
# look at tail
fives_ones.tail()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
9990,R8VwdLyvsp9iybNqRvm94g,2011-10-03,pcEeHdAJPoFNF23es0kKWg,5,Yes I do rock the hipster joints. I dig this ...,review,b92Y3tyWTQQZ5FLifex62Q,1,1,1
9991,WJ5mq4EiWYAA4Vif0xDfdg,2011-12-05,EuHX-39FR7tyyG1ElvN1Jw,5,Only 4 stars? \n\n(A few notes: The folks that...,review,hTau-iNZFwoNsPCaiIUTEA,1,1,0
9992,f96lWMIAUhYIYy9gOktivQ,2009-03-10,YF17z7HWlMj6aezZc-pVEw,5,I'm not normally one to jump at reviewing a ch...,review,W_QXYA7A0IhMrvbckz7eVg,2,3,2
9994,L3BSpFvxcNf3T_teitgt6A,2012-03-19,0nxb1gIGFgk3WbC5zwhKZg,5,Let's see...what is there NOT to like about Su...,review,OzOZv-Knlw3oz9K5Kh5S6A,1,2,1
9999,pF7uRzygyZsltbmVpjIyvw,2010-10-16,vWSmOhg2ID1MNZHaWapGbA,5,4-5 locations.. all 4.5 star average.. I think...,review,KSBFytcdjPKZgXKQnYQdkA,0,0,0


In [154]:
# Check the value counts to make sure only contains 5 and 1 classes
fives_ones.stars.value_counts()

5    3337
1     749
Name: stars, dtype: int64

###Task 3
Define X and y from the new DataFrame, and then split X and y into training and testing sets, using the review text as the only feature and the star rating as the response.

- Hint: Keep in mind that X should be a Pandas Series (not a DataFrame), since we will pass it to CountVectorizer in the task that follows.

In [155]:
# create X feature series of text reviews ONLY
# create y response series of star ratings
X = fives_ones.text 
y= fives_ones.stars

In [156]:
# splite these two Series into train and test splits
# use the sklearn cross validation package train_test_split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

In [157]:
# Check the shape of each of the new train and test splits
print "X_train", X_train.shape 
print "y_train", y_train.shape
print
print "X_test", X_test.shape
print "y_test", y_test.shape

X_train (3064,)
y_train (3064,)

X_test (1022,)
y_test (1022,)


In [158]:
X.head()

0    My wife took me here on my birthday for breakf...
1    I have no idea why some people give bad review...
3    Rosie, Dakota, and I LOVE Chaparral Dog Park!!...
4    General Manager Scott Petello is a good egg!!!...
6    Drop what you're doing and drive here. After I...
Name: text, dtype: object

In [159]:
y.head()

0    5
1    5
3    5
4    5
6    5
Name: stars, dtype: int64

### Task 4
Use CountVectorizer to create document-term matrices from X_train and X_test.
- CountVectorizer is a feature extraction tool in SKL 
- CountVectorizer converts a collection of text documents into a matrix of token counts
- Each document (ie, observation) passed to the CountVectorizer algorithm will be parsed and each word will be converted to a matrix where each word is a feature in that matrix along with a count of the number of times that word appeared in the document.
- The result is a sparse feature matrix
- This process is known as vectorization (as well as a tokenization) and results in what is commonly referred to as a bag of words.

In [160]:
# import the CountVectorizer
# instantiate the CountVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [161]:
# The CountVecorizer instance learns the X training data vocabulary
vect.fit(X_train)

# This learned vocabular is then used to create the document term matrix
# This docment term matrix (dtm) is the sparse matrix where each word is a feature
# And each row contains the count of that word contained in that observations document.
X_train_dtm = vect.transform(X_train)

#NB: These steps can be combined
#X_train_dtm = vect.fit_transform(X_train)

In [162]:
# examine the document term matrix
print X_train_dtm
X_train_dtm

  (0, 136)	1
  (0, 805)	4
  (0, 1003)	1
  (0, 1018)	1
  (0, 1528)	1
  (0, 1564)	1
  (0, 1902)	1
  (0, 2069)	1
  (0, 2285)	1
  (0, 2286)	2
  (0, 2398)	1
  (0, 2789)	1
  (0, 3213)	1
  (0, 3233)	1
  (0, 3679)	1
  (0, 3999)	1
  (0, 4631)	1
  (0, 4809)	1
  (0, 4837)	1
  (0, 5342)	2
  (0, 5773)	1
  (0, 5882)	1
  (0, 6028)	1
  (0, 6154)	1
  (0, 6433)	1
  :	:
  (3063, 6974)	1
  (3063, 7023)	1
  (3063, 7181)	1
  (3063, 8189)	2
  (3063, 9318)	1
  (3063, 9438)	1
  (3063, 9440)	1
  (3063, 9807)	1
  (3063, 10352)	2
  (3063, 10805)	1
  (3063, 12038)	1
  (3063, 14137)	1
  (3063, 14720)	1
  (3063, 14836)	1
  (3063, 14994)	1
  (3063, 15032)	2
  (3063, 15042)	1
  (3063, 15093)	2
  (3063, 15228)	1
  (3063, 15281)	1
  (3063, 15834)	1
  (3063, 15968)	1
  (3063, 16162)	1
  (3063, 16252)	2
  (3063, 16599)	1


<3064x16825 sparse matrix of type '<type 'numpy.int64'>'
	with 237720 stored elements in Compressed Sparse Row format>

In [163]:
# transform the testing data, using the fitted vocab in vect
# the result is that any new words in the X_test set will be ignored
X_test_dtm = vect.transform(X_test)

In [164]:
# examine the X_test_dtm document term matrix
X_test_dtm

#Notice that it has the same number of unique words (features) as the X_train  dat set
# However it still has the same number of observations as X_test

<1022x16825 sparse matrix of type '<type 'numpy.int64'>'
	with 77006 stored elements in Compressed Sparse Row format>

In [165]:
print vect

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


###Task 5
- Use Multinomial Naive Bayes to predict the star rating for the reviews in the testing set, and then calculate the accuracy and print the confusion matrix.
- Hint: Evaluating a classification model explains how to interpret both classification accuracy and the confusion matrix.

In [166]:
# import the Multinomial NB classifier from SKL
from sklearn.naive_bayes import MultinomialNB

# instantiate the Multi NB classifier
nb = MultinomialNB()

In [167]:
# train the model using the X_train_dtm (and time with %time magic function)
%time nb.fit(X_train_dtm, y_train)

CPU times: user 6.92 ms, sys: 2.03 ms, total: 8.95 ms
Wall time: 8.02 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [168]:
# make class predictions on the reviews based on the X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [169]:
# calculate the accuracy of the class predictions
from sklearn import metrics
metrics.accuracy_score(y_test,y_pred_class)

0.91878669275929548

In [170]:
# print the confusion matrix
metrics.confusion_matrix(y_test,y_pred_class)

array([[126,  58],
       [ 25, 813]])

In [171]:
# putll out the associated predicted probability of each review
y_pred_prob = nb.predict_proba(X_test_dtm)[:,1]
y_pred_prob

array([  1.00000000e+00,   9.99759874e-01,   1.00000000e+00, ...,
         1.00000000e+00,   1.05200807e-19,   9.99915316e-01])

###Task 6 (Challenge)
Calculate the null accuracy, which is the classification accuracy that could be achieved by always predicting the most frequent class.

In [172]:
# examine the class distributions 
y_test.value_counts()

# Null Accuracy would be simply always choosing 5 as the response value

5    838
1    184
Name: stars, dtype: int64

In [173]:
# calculate the percentages of each class
fives = y_test.value_counts()[5]
ones = y_test.value_counts()[1]
total = fives + ones

In [174]:
# Calculate the percent of y_test responses that are 5 stars
five_acc = float(fives)/total
one_acc = float(ones)/total
print "Null accuracy null_acc is", max(five_acc, one_acc)

Null accuracy null_acc is 0.819960861057


###Task 7 (Challenge)
Browse through the review text of some of the false positives and false negatives. Based on your knowledge of how Naive Bayes works, do you have any ideas about why the model is incorrectly classifying these reviews?

In [175]:
# False Positive (FP): an actual one star review was predicted as a five star review, 
# ie y_test < y_pred_class
# False Negative (FN): an acutal five star review was predicted as a one star review, 
# ie y_test > y_pred_class

FP = y_test < y_pred_class
FN = y_test > y_pred_class

In [176]:
# Create variables containing those reviews that are FN and FP respectively to examine a few
FP_reviews = X_test[FP]
FN_reviews = X_test[FN]

In [177]:
# Examine the head of the FP reviews to check out which review ids to look at.
FP_reviews.head()

2175    This has to be the worst restaurant in terms o...
1781    If you like the stuck up Scottsdale vibe this ...
2674    I'm sorry to be what seems to be the lone one ...
9984    Went last night to Whore Foods to get basics t...
3392    I found Lisa G's while driving through phoenix...
Name: text, dtype: object

In [178]:
# Review a few of these False Positives 
FP_reviews[2175]
FP_reviews[1781]
FP_reviews[2674]
FP_reviews[9984]
FP_reviews[3392]

"I found Lisa G's while driving through phoenix with a friend. The exterior looked promising and we prepared ourselves for a treat. We were in for a a shock! We ordered the antipasto plate and while there were serious issues with it, it turned out to be the best thing we were served. I ordered the chicken rolatini (the special of the day) and it was just plain bad! The chicken was dry and under seasoned, the Rosemary potatoes were soggy  and the veggie medley  (squash, corn, green beans, etc.) was over cooked and the juices spilled over into the potatoes. It was one of the worst $17 meals that I have ever had! My friend ordered the veggie salad and it was one literal hot mess. Just horrible food! I see that others had the mini burgers and bowl of balls and found them to be tasty. I wish I had ordered those. As it is this is one place I would never revisit or recommend to a friend!"

In [179]:
# Examine the head of the False Negative Reviews
FN_reviews.head()

7148    I now consider myself an Arizonian. If you dri...
4963    This is by far my favourite department store, ...
6318    Since I have ranted recently on poor customer ...
380     This is a must try for any Mani Pedi fan. I us...
5565    I`ve had work done by this shop a few times th...
Name: text, dtype: object

In [180]:
# Review the content of a handful of FN reviews to better understand their misclassification
FN_reviews[7148]
FN_reviews[4963]

'This is by far my favourite department store, hands down. I have had nothing but perfect experiences in this store, without exception, no matter what department I\'m in. The shoe SA\'s will bend over backwards to help you find a specific shoe, and the staff will even go so far as to send out hand-written thank you cards to your home address after you make a purchase - big or small. Tim & Anthony in the shoe salon are fabulous beyond words! \n\nI am not completely sure that I understand why people complain about the amount of merchandise on the floor or the lack of crowds in this store. Frankly, I would rather not be bombarded with merchandise and other people. One of the things I love the most about Barney\'s is not only the prompt attention of SA\'s, but the fact that they aren\'t rushing around trying to help 35 people at once. The SA\'s at Barney\'s are incredibly friendly and will stop to have an actual conversation, regardless or whether you are purchasing something or not. I hav

### Based on your knowledge of how Naive Bayes works, do you have any ideas about why the model is incorrectly classifying these reviews?
**The model is incorrectly classifying these FP and FN reviews because of the simplicity of the classifier which only looks at unique  words and their respective counts when determining likely association with a given response class.  This results in broader context and subtlety of speech being misinterpreted by the classifier - for instance with sarcasm or negation as well as ambiguity with words that have multiple meanings in a language.** 
- For example, **the classifier cannot incorporate the impact of negation**: The phrase, *"I don't like this place"* could be seen as a positive review by the classifier because it will count the 1-gram "like" as contributing to the increased likilehood this a positive review.  
- Additionally with **word ambiguity**:  A review that uses the word "like" as a preposition to mean "similiar to" (ie, *"the bun tasted like liver."*) will count the presence of the word "like" towards an increased likelihood this is a positive review because the classifier has learned that "like" is most often present in positive reviews which employ "like" as a verb.


##Task 8 (Challenge)
Calculate which 10 tokens are the most predictive of 5-star reviews, and which 10 tokens are the most predictive of 1-star reviews.
Hint: Naive Bayes automatically counts the number of times each token appears in each class, as well as the number of observations in each class. You can access these counts via the feature_count_ and class_count_ attributes of the Naive Bayes model object.

In [181]:
# store the vocabulary of X_train and review the number of tokens
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

16825

In [182]:
# Examine a few of the first tokens
print X_train_tokens[350:400]

[u'absoloutely', u'absolut', u'absolute', u'absolutely', u'absolutley', u'absolutly', u'abstained', u'absurd', u'abuelo', u'abundance', u'abundant', u'abuse', u'abused', u'abusive', u'ac', u'academy', u'acapulco', u'accent', u'accented', u'accents', u'accept', u'acceptable', u'accepted', u'access', u'accessed', u'accessible', u'accessories', u'accessorize', u'accessory', u'accident', u'accidental', u'accidentally', u'accolades', u'accommodate', u'accommodated', u'accommodates', u'accommodating', u'accommodations', u'accomodate', u'accomodating', u'accompanied', u'accompanies', u'accompaniment', u'accompany', u'accompanying', u'accomplish', u'accomplished', u'accomplishment', u'according', u'accordingly']


In [183]:
# Examine the last 50 tokens
print X_train_tokens[-50:]

[u'yyyyy', u'z11', u'za', u'zabba', u'zach', u'zam', u'zanella', u'zankou', u'zappos', u'zatsiki', u'zen', u'zero', u'zest', u'zexperience', u'zha', u'zhou', u'zia', u'zihuatenejo', u'zilch', u'zin', u'zinburger', u'zinburgergeist', u'zinc', u'zinfandel', u'zing', u'zip', u'zipcar', u'zipper', u'zippers', u'zipps', u'ziti', u'zoe', u'zombi', u'zombies', u'zone', u'zones', u'zoning', u'zoo', u'zoyo', u'zucca', u'zucchini', u'zuchinni', u'zumba', u'zupa', u'zuzu', u'zwiebel', u'zzed', u'\xe9clairs', u'\xe9cole', u'\xe9m']


In [184]:
# Review the number of times each token appears in each class
nb.feature_count_

array([[ 26.,   4.,   1., ...,   0.,   0.,   0.],
       [ 39.,   5.,   0., ...,   1.,   1.,   1.]])

In [185]:
# grab the number of times each token appears across all reviews in the X_Train dataset
one_token_count = nb.feature_count_[0, :]
five_token_count = nb.feature_count_[1, :]
print "One tokens:", one_token_count
print "Five tokens:", five_token_count

One tokens: [ 26.   4.   1. ...,   0.   0.   0.]
Five tokens: [ 39.   5.   0. ...,   1.   1.   1.]


In [186]:
# Create a DataFrame of the raw token counts separated with columns for one and five stars
tokens = pd.DataFrame({"token":X_train_tokens,"one": one_token_count, "five":five_token_count}).set_index('token')

In [187]:
# Examine random tokens to see counts of each by class
tokens.sample(5, random_state=2)

,five,one
token,,
puree,5.0,0.0
rama,1.0,0.0
helps,8.0,1.0
forgotten,10.0,2.0
farther,1.0,0.0


In [188]:
# Add 1 and normalize counts of each token to determine 10 most predictive tokens for one star reviews and 
# Five star reviews respectively
tokens['five'] = tokens.five + 1
tokens['one'] = tokens.one + 1

In [189]:
tokens.sample(5,random_state=2)

,five,one
token,,
puree,6.0,1.0
rama,2.0,1.0
helps,9.0,2.0
forgotten,11.0,3.0
farther,2.0,1.0


In [190]:
# Check the count of each class
print "One Count:", int(nb.class_count_[0])
print "Five Count:", int(nb.class_count_[1])
#print "5 Star Count:", len(y_train[y_train == 5])

One Count: 565
Five Count: 2499


In [191]:
# convert the five and one star token counts into frequencies
# NB: Only run this cell once or the count per token will be divided by its class count each time
tokens['one'] = tokens.one / nb.class_count_[0]
tokens['five'] = tokens.five / nb.class_count_[1]

In [192]:
# examine the class frequency of each token
tokens.sample(5,random_state=2)

,five,one
token,,
puree,0.002401,0.00177
rama,0.000800,0.00177
helps,0.003601,0.00354
forgotten,0.004402,0.00531
farther,0.000800,0.00177


In [193]:
# Calculate the ratio of one-to-five star reviews for each token
tokens['one_star_ratio'] = tokens.one/tokens.five

In [194]:
# Review the likelihood each token is associated with a 1 star review
tokens.sample(5,random_state=2)

,five,one,one_star_ratio
token,,,
puree,0.002401,0.00177,0.737168
rama,0.000800,0.00177,2.211504
helps,0.003601,0.00354,0.982891
forgotten,0.004402,0.00531,1.206275
farther,0.000800,0.00177,2.211504


### Top 10 tokens most predicitve of 1-star reviews

In [195]:
# Examine the DF sorted by one_star_ratio
# Sort in descending order to find the top 10 tokens most predictive of 1-star reviews
tokens.sort_values('one_star_ratio',ascending = False).head(10)

,five,one,one_star_ratio
token,,,
staffperson,0.0004,0.030088,75.191150
refused,0.0004,0.024779,61.922124
disgusting,0.0008,0.042478,53.076106
filthy,0.0004,0.019469,48.653097
unacceptable,0.0004,0.015929,39.807080
acknowledge,0.0004,0.015929,39.807080
unprofessional,0.0004,0.015929,39.807080
ugh,0.0008,0.030088,37.595575
yuck,0.0008,0.028319,35.384071


### Top 10 tokens most predicitve of 5-star reviews

In [196]:
# Examine the DF sorted by one_star_ratio
# Sort in descending order to find the top 10 tokens most predictive of 1-star reviews
tokens.sort_values('one_star_ratio').head(10)

,five,one,one_star_ratio
token,,,
fantastic,0.077231,0.003540,0.045834
perfect,0.098039,0.005310,0.054159
yum,0.024810,0.001770,0.071339
favorite,0.138055,0.012389,0.089742
outstanding,0.019608,0.001770,0.090265
brunch,0.016807,0.001770,0.105310
gem,0.016006,0.001770,0.110575
mozzarella,0.015606,0.001770,0.113410
pasty,0.015606,0.001770,0.113410



###Task 9 (Challenge)
Up to this point, we have framed this as a binary classification problem by only considering the 5-star and 1-star reviews. Now, let's repeat the model building process using all reviews, which makes this a 5-class classification problem.

In [197]:
# Create X feature matrix and y response series using all observations in the dataset
X_all = posts.text
y_all = posts.stars
print X_all.shape
print y_all.shape

(10000,)
(10000,)


In [198]:
# Create train_test split
X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(X_all, y_all, random_state = 1)
print X_all_train.shape
print X_all_test.shape
print y_all_train.shape
print y_all_test.shape

(7500,)
(2500,)
(7500,)
(2500,)


In [199]:
# instantiate the CV
vect_all = CountVectorizer()
# fit_transform the vect_all CV and create DTMs
X_train_all_dtm = vect_all.fit_transform(X_all_train)
X_test_all_dtm = vect_all.transform(X_all_test)

# print details X_train_all_dtm
print X_train_all_dtm
X_train_all_dtm

  (0, 10754)	1
  (0, 13443)	1
  (0, 11649)	2
  (0, 4401)	1
  (0, 9216)	3
  (0, 14935)	2
  (0, 15765)	2
  (0, 15116)	2
  (0, 13287)	1
  (0, 24893)	1
  (0, 14895)	1
  (0, 23005)	1
  (0, 15333)	1
  (0, 25080)	1
  (0, 1238)	1
  (0, 1577)	1
  (0, 17145)	3
  (0, 8353)	1
  (0, 25428)	1
  (0, 2332)	1
  (0, 13321)	1
  (0, 23029)	3
  (0, 16507)	1
  (0, 1741)	1
  (0, 25045)	1
  :	:
  (7499, 7245)	1
  (7499, 23814)	1
  (7499, 13104)	1
  (7499, 20132)	1
  (7499, 18560)	1
  (7499, 22453)	1
  (7499, 10075)	1
  (7499, 11774)	1
  (7499, 11271)	1
  (7499, 21208)	1
  (7499, 22200)	1
  (7499, 12313)	1
  (7499, 9250)	1
  (7499, 11559)	1
  (7499, 3474)	1
  (7499, 21048)	1
  (7499, 18639)	1
  (7499, 19493)	1
  (7499, 5465)	1
  (7499, 12693)	1
  (7499, 1849)	1
  (7499, 22367)	1
  (7499, 23648)	1
  (7499, 4890)	1
  (7499, 123)	1


<7500x25797 sparse matrix of type '<type 'numpy.int64'>'
	with 622700 stored elements in Compressed Sparse Row format>

In [200]:
# print details about X_test_all_dtm
print X_test_all_dtm
X_test_all_dtm

  (0, 1262)	1
  (0, 1829)	1
  (0, 2020)	1
  (0, 2046)	1
  (0, 2246)	1
  (0, 2332)	1
  (0, 3578)	1
  (0, 5904)	1
  (0, 6879)	1
  (0, 8760)	1
  (0, 9942)	1
  (0, 10052)	1
  (0, 10249)	1
  (0, 11047)	1
  (0, 11062)	1
  (0, 11649)	1
  (0, 12187)	1
  (0, 12540)	1
  (0, 13612)	1
  (0, 13982)	1
  (0, 14379)	1
  (0, 15415)	1
  (0, 15565)	1
  (0, 15765)	1
  (0, 15832)	1
  :	:
  (2499, 23029)	24
  (2499, 23058)	1
  (2499, 23072)	2
  (2499, 23089)	1
  (2499, 23112)	1
  (2499, 23317)	5
  (2499, 23380)	5
  (2499, 23390)	1
  (2499, 23509)	1
  (2499, 23684)	1
  (2499, 23758)	1
  (2499, 23759)	1
  (2499, 23760)	1
  (2499, 24359)	1
  (2499, 24480)	1
  (2499, 24573)	3
  (2499, 24827)	2
  (2499, 24852)	1
  (2499, 24893)	4
  (2499, 25309)	2
  (2499, 25314)	1
  (2499, 25405)	1
  (2499, 25428)	1
  (2499, 25641)	3
  (2499, 25653)	3


<2500x25797 sparse matrix of type '<type 'numpy.int64'>'
	with 200729 stored elements in Compressed Sparse Row format>

In [201]:
# Run Multinomial NB classifier to create model and make predictions
#instantiate
nb_all = MultinomialNB()
# fit
nb.fit(X_train_all_dtm, y_all_train)
# predict
y_pred_all = nb.predict(X_test_all_dtm)

In [202]:
# Examine accuracy of the predicted values
# import the metrics module from SKL
from sklearn import metrics
metrics.accuracy_score(y_all_test,y_pred_all)

0.47120000000000001

In [203]:
# Calculate null accuracy
# Determine raw counts of each 
print y_all_train.value_counts()

# Convert raw counts to percentages and create dictionary of class:percent
ratings = range(1,6)
d = {x:((y_all_train.value_counts()[x])/(float(len(y_all_train)))) for x in ratings}
null_acc = max(d.values())

# Find the null accuracy and the associated class
null_star = [k for k,v in d.iteritems() if v == null_acc]
print "The null accuracy is", str(round(null_acc,3)*100) + "% " + "and is for class", null_star[0]

4    2642
5    2505
3    1096
2     693
1     564
Name: stars, dtype: int64
The null accuracy is 35.2% and is for class 4


In [204]:
# Review the value_count class details about y_all_test 
print y_all_test.shape[0]
print y_all_test.value_counts()
print y_all_test.value_counts(1)

2500
4    884
5    832
3    365
2    234
1    185
Name: stars, dtype: int64
4    0.3536
5    0.3328
3    0.1460
2    0.0936
1    0.0740
Name: stars, dtype: float64


In [205]:
# Print the confusion matrix
metrics.confusion_matrix(y_all_test,y_pred_all)

array([[ 55,  14,  24,  65,  27],
       [ 28,  16,  41, 122,  27],
       [  5,   7,  35, 281,  37],
       [  7,   0,  16, 629, 232],
       [  6,   4,   6, 373, 443]])

###Comment on the output of the confusion matrix
Overall, the classifier demonstrates a classification accuracy of 47%, which, while not great, is 12 points better than the null accuracy of 35.2%.  In general, the model skews towards predicting a "positive" review, i.e. designating a review as either class 4 or class 5 - with a particular tendency to predict class 4.  For example, in this split, the model predicted 1470 class 4 responses out of a total of 2500 observations.  This represents ~ 59% of all reviews, despite the fact that class 4 responses only constitute about 35% of the actual observations.  In fact, class 4 is the most predicted class across actual classes, with the exception of class 5.  It is model's ability to accuartely predict class 4 (approx 70% accuracy) and class 5 (approx 50% accuracy) that adds the most predictive power to the model as compared to the null accuracy.  

In contrast, with neutral (3 star) and "negative" (1 or 2 star) classes, the model does not have much predictive power.  In the cases of an actual class 1, the model does correctly predict 30% of observations (55 reviews) accurately despite only 7.5% of the test observations belonging to Class 1.  However, the model still incorrectly labels 65 actual Class 1 observatins as belonging to Class 4, which is still greater in raw terms than the 55 observations it correctly assigns to class 1. And the model is particularly poor at discerning Class 2 observations (16 total correct, or 6.8% accuracy) and Class 3 observations (35 total correct, or ~10% accuracy).

In [206]:
# Import the Clasification Report from the SKL metrics module
from sklearn.metrics import classification_report

In [207]:
# Run the Classification Report on the y_test and y_pred_prob classes
class_report = classification_report(y_all_test, y_pred_all)

In [208]:
# print the results of the clasification report
print class_report

             precision    recall  f1-score   support

          1       0.54      0.30      0.38       185
          2       0.39      0.07      0.12       234
          3       0.29      0.10      0.14       365
          4       0.43      0.71      0.53       884
          5       0.58      0.53      0.55       832

avg / total       0.46      0.47      0.43      2500



### Comment on the Results of the Classification Report
The model demonstrates an average precision across all classes of 46% along with an average recall of 47%.  Overall, the weighted average of these two scores results in the model having an f1-score, or an average accuracy, of 43%.  This is still better than the null accuracy that can be achieved by simply labeling all observations as class 4, but there are some areas where the model could be improved.  Particularly, the model's recall on Class 1, 2 and 3 is further evidence that this model is particually poor at predicting non-positive (ie, non Class 4 or 5) reviews, as noted in the confusion matrix analysis above.  Additionally, the model's high recall and low precision with Class 4 indicates the model's skewed tendency towards labeling a majority of observations as Class 4.  Based on these findings, I would suggest improving the model (perhaps by parameter tuning or the use of a larger datset) to better discern features that are specific to non-positive classes (Classes 1,2,3) and perhaps using the model to predict positive vs non-positive ratings instead.